# Perf Tools Analisys

Neste notebook iremos verificar as analises referentes ao perf tools da implementação do loop fusion, verificando quanto de acesso ao cache misses e cache references é utilizado.

In [150]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import datetime
import time
from IPython.display import display
from collections import Counter
import os
import re
import math
import random
pd.set_option('display.max_rows', 300)
pd.options.display.float_format = '{:,.2f}'.format

In [151]:
#loads the csv as a dataframe and standardizes the algorithm names 
def load_df(filename):
    df = pd.read_csv(filename, index_col=False)
    return select_columns_and_rename_values(df)

In [152]:
#filters by substring (there are multiple OzaBag algorithms)
def filter_by_substring_algorithm(df, string):
    aux = df[df['algorithm'].str.contains(string, regex=False)]
    ret = aux
    if string == 'OB':
        ret = aux[~aux.algorithm.str.contains("Adwin|ASHT")]
    elif string == 'OzaBag':
        ret = aux[(aux.algorithm.str.contains(string)) & (~aux.algorithm.str.contains("Adwin|ASHT"))]
    return ret

In [214]:
#standardize algorithm names
def select_columns_and_rename_values(df):
    df.algorithm = df.algorithm.str.replace("Executor", "")
    df['algorithm'] = df["algorithm"].str.replace("OzaBag", "OB")
    df['algorithm'] = df["algorithm"].str.replace("AdaptiveRandomForest", "ARF")
    df['algorithm'] = df["algorithm"].str.replace("SequentialChunk", "SeqMB")
    df['algorithm'] = df["algorithm"].str.replace("OB$", "OBSequential")
    df['algorithm'] = df['algorithm'].str.replace("LeveragingBag", "LBagSequential")
    df['algorithm'] = df['algorithm'].str.replace("Adwin$", "AdwinSequential")
    df['algorithm'] = df['algorithm'].str.replace("CHUNK", "MB")
    df['algorithm'] = df['algorithm'].str.replace("MAXChunk", "MB")
    df['algorithm'] = df['algorithm'].str.replace("StreamingRandomPatches", "SRP")
    df['algorithm'] = df['algorithm'].str.replace("SRP$", "SRPSequential")
    df['algorithm'] = df['algorithm'].str.replace("OBASHT$", "OBASHTSequential")
    df.batch_size.unique()
    return df

In [229]:
#Folder inside results directory that contains all the MOA dump files for these experiments
%cd /Users/reginaldoluisdeluna/Documents/Ufscar/comparison-xue3m-minibatching/
folderMOADumps = "/Users/reginaldoluisdeluna/Documents/Ufscar/comparison-xue3m-minibatching/results/sem-coletor/mini-batching/perf/first"
wantedCSVfilename = "loop-fusion-perf"

/Users/reginaldoluisdeluna/Documents/Ufscar/comparison-xue3m-minibatching


In [230]:
import re
MILLION = 1
def parse_perf_file(fname, methodology='MB'):
    spname = fname.split('/')[-1].split('-')
    
    cores = 4
    if methodology == 'Sequential':
        cores = 1

    lineDict = {
        'dataset': spname[1],
        'algorithm': spname[2],
        'ensemble': spname[3],
        'methodology': methodology,
        'cores': cores,
        'batch_size': spname[5],
        'rate': spname[6],
        'cache-misses': None,
        'cache-references': None
    }

    with open (fname) as file:
        cache_misses = "0"
        cache_references = "0"
        for line in file:
            line = re.findall(r'\S+', line)
            try:
                if line[1]:
                    if line[1] == "cache-misses:u":
                        cache_misses = line[0]
                        
                    if line[1] == "cache-references:u":
                        cache_references = line[0]
    
                    response.append(lineDict)
            except IndexError:
                pass
    
    lineDict['cache-misses'] = float(cache_misses.replace(',', '')) / MILLION
    lineDict['cache-references'] = float(cache_references.replace(',', '')) / MILLION
            
    return lineDict

In [231]:
resultsFolder = f"{folderMOADumps}"
csvFile = f"parsed_csvs/{wantedCSVfilename}"
directory = os.fsencode(resultsFolder)
header_printed = False

response_mb = []
with open(f"{csvFile}", "w+") as output:
    for file in os.listdir(directory):
        filename = os.fsdecode(file)
        if filename.startswith("perf-"):
            response_mb.append(
                parse_perf_file(f'{os.fsdecode(directory)}/{filename}', 'MB')
            )

df_mini_batching = pd.DataFrame.from_dict(response_mb)
display(df_mini_batching.sort_values(by=['dataset', 'algorithm']).drop_duplicates())

,dataset,algorithm,ensemble,methodology,cores,batch_size,rate,cache-misses,cache-references
17,GMSC,AdaptiveRandomForestExecutorMAXChunk,25,MB,4,50,1,"1,754,556,106.00","66,333,153,316.00"
9,GMSC,LBagExecutorMAXChunk,25,MB,4,50,1,"1,445,816,307.00","55,310,768,620.00"
12,GMSC,OzaBagASHTExecutorMAXChunk,25,MB,4,50,1,"568,482,984.00","17,763,953,604.00"
1,GMSC,OzaBagAdwinExecutorMAXChunk,25,MB,4,50,1,"554,639,590.00","22,920,631,222.00"
21,GMSC,OzaBagExecutorMAXChunk,25,MB,4,50,1,"655,211,863.00","18,911,127,977.00"
18,GMSC,StreamingRandomPatchesExecutorMAXChunk,25,MB,4,50,1,"2,072,185,656.00","75,241,199,698.00"
22,airlines,AdaptiveRandomForestExecutorMAXChunk,25,MB,4,50,1,0.00,0.00
7,airlines,LBagExecutorMAXChunk,25,MB,4,50,1,"2,334,172,813.00","93,299,545,022.00"
23,airlines,OzaBagASHTExecutorMAXChunk,25,MB,4,50,1,"1,850,163,989.00","85,953,636,884.00"
3,airlines,OzaBagAdwinExecutorMAXChunk,25,MB,4,50,1,"2,199,843,321.00","92,043,681,856.00"


In [236]:
#Folder inside results directory that contains all the MOA dump files for these experiments
%cd /Users/reginaldoluisdeluna/Documents/Ufscar/comparison-xue3m-minibatching/
folderMOADumps = "/Users/reginaldoluisdeluna/Documents/Ufscar/comparison-xue3m-minibatching/results/loop-fusion/loop-fusion-perf/second"
wantedCSVfilename = "loop-fusion-perf"

/Users/reginaldoluisdeluna/Documents/Ufscar/comparison-xue3m-minibatching


In [237]:
resultsFolder = f"{folderMOADumps}"
csvFile = f"parsed_csvs/{wantedCSVfilename}"
directory = os.fsencode(resultsFolder)
header_printed = False

response = []
with open(f"{csvFile}", "w+") as output:
    for file in os.listdir(directory):
        filename = os.fsdecode(file)
        if filename.startswith("perf-"):
            response.append(
                parse_perf_file(f'{os.fsdecode(directory)}/{filename}', 'MB-LF')
            )

df = pd.DataFrame.from_dict(response)
display(df.sort_values(by=['dataset', 'algorithm']).drop_duplicates())

,dataset,algorithm,ensemble,methodology,cores,batch_size,rate,cache-misses,cache-references
464,GMSC,AdaptiveRandomForestExecutorMAXChunk,25,MB-LF,4,50,1,"1,854,566,334.00","72,717,007,187.00"
145,GMSC,LBagExecutorMAXChunk,25,MB-LF,4,50,1,"1,378,595,612.00","57,385,891,110.00"
29,GMSC,OzaBagASHTExecutorMAXChunk,25,MB-LF,4,50,1,"360,425,670.00","12,057,654,946.00"
290,GMSC,OzaBagAdwinExecutorMAXChunk,25,MB-LF,4,50,1,"547,883,621.00","23,702,321,595.00"
523,GMSC,OzaBagExecutorMAXChunk,25,MB-LF,4,50,1,"400,553,855.00","12,859,901,006.00"
435,GMSC,StreamingRandomPatchesExecutorMAXChunk,25,MB-LF,4,50,1,"3,656,737,167.00","141,587,308,380.00"
522,airlines,AdaptiveRandomForestExecutorMAXChunk,25,MB-LF,4,50,1,0.00,0.00
58,airlines,LBagExecutorMAXChunk,25,MB-LF,4,50,1,"61,329,640,613.00","2,574,339,121,768.00"
493,airlines,OzaBagASHTExecutorMAXChunk,25,MB-LF,4,50,1,"1,752,395,082.00","150,673,108,970.00"
261,airlines,OzaBagAdwinExecutorMAXChunk,25,MB-LF,4,50,1,"17,816,007,434.00","820,965,909,736.00"


In [234]:
#Folder inside results directory that contains all the MOA dump files for these experiments
%cd /Users/reginaldoluisdeluna/Documents/Ufscar/comparison-xue3m-minibatching/
folderMOADumps = "/Users/reginaldoluisdeluna/Documents/Ufscar/comparison-xue3m-minibatching/results/loop-fusion/sequential-perf/first"
wantedCSVfilename = "sequential-perf"

/Users/reginaldoluisdeluna/Documents/Ufscar/comparison-xue3m-minibatching


In [235]:
resultsFolder = f"{folderMOADumps}"
csvFile = f"parsed_csvs/{wantedCSVfilename}"
directory = os.fsencode(resultsFolder)
header_printed = False

response = []
with open(f"{csvFile}", "w+") as output:
    for file in os.listdir(directory):
        filename = os.fsdecode(file)
        if filename.startswith("perf-"):
            response.append(
                parse_perf_file(f'{os.fsdecode(directory)}/{filename}', 'Sequential')
            )

df = pd.DataFrame.from_dict(response)
display(df.sort_values(by=['dataset', 'algorithm']).drop_duplicates())

,dataset,algorithm,ensemble,methodology,cores,batch_size,rate,cache-misses,cache-references
580,GMSC,AdaptiveRandomForestSequential,25,Sequential,1,1,1,"1,061,924,674.00","35,398,625,129.00"
174,GMSC,LeveragingBag,25,Sequential,1,1,1,"1,045,036,877.00","32,457,902,246.00"
203,GMSC,OzaBag,25,Sequential,1,1,1,"716,212,401.00","16,950,503,006.00"
377,GMSC,OzaBagASHT,25,Sequential,1,1,1,"701,863,572.00","17,968,039,181.00"
522,GMSC,OzaBagAdwin,25,Sequential,1,1,1,"828,723,368.00","22,116,552,548.00"
406,GMSC,StreamingRandomPatches,25,Sequential,1,1,1,"1,109,616,588.00","37,295,121,000.00"
290,airlines,AdaptiveRandomForestSequential,25,Sequential,1,1,1,"1,152,112,660.00","48,376,350,528.00"
493,airlines,LeveragingBag,25,Sequential,1,1,1,"1,306,807,180.00","46,331,210,615.00"
0,airlines,OzaBag,25,Sequential,1,1,1,"1,224,424,255.00","45,372,623,382.00"
58,airlines,OzaBagASHT,25,Sequential,1,1,1,"1,158,245,479.00","45,298,745,971.00"
